# Quiz 8

In [3]:
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance
import pandas as pd

In [8]:
h81 = pd.read_csv("homework_8.1.csv").drop("Unnamed: 0", axis=1)
h81

,X,Y,Z
0,1,4.109218,1.764052
1,0,2.259504,0.400157
2,0,-0.647584,0.978738
3,0,2.106071,2.240893
4,1,3.583464,1.867558
...,...,...,...
995,1,1.757928,0.412871
996,1,0.631199,-0.198399
997,0,1.104620,0.094192
998,0,-0.651432,-1.147611


In [13]:
# load data
data = pd.read_csv("homework_8.1.csv")

# tell python which columns are which
treatment = data["X"]        # 1 = treated, 0 = control
outcome = data["Y"]          # outcome
covariates = data.drop(columns=["X", "Y"])  # stuff we adjust for (Z's)

# step 1: estimate propensity scores P(X=1 | Z)
log_model = LogisticRegression()
log_model.fit(covariates, treatment)
propensity_scores = log_model.predict_proba(covariates)[:, 1]

# step 2: make inverse probability weights
weights = pd.Series(0.0, index=data.index)
weights[treatment == 1] = 1.0 / propensity_scores[treatment == 1]
weights[treatment == 0] = 1.0 / (1.0 - propensity_scores[treatment == 0])

# step 3: weighted means for treated and control
weighted_y_treated = (outcome[treatment == 1] * weights[treatment == 1]).sum() / weights[treatment == 1].sum()
weighted_y_control = (outcome[treatment == 0] * weights[treatment == 0]).sum() / weights[treatment == 0].sum()

# step 4: ATE = difference
ate_ipw = weighted_y_treated - weighted_y_control

print("Weighted treated mean:", weighted_y_treated)
print("Weighted control mean:", weighted_y_control)
print("ATE (IPW):", ate_ipw)

Weighted treated mean: 2.2303100401962888
Weighted control mean: -0.04116885000749054
ATE (IPW): 2.2714788902037792


In [14]:
propensity_scores[0:10]

array([0.80770931, 0.52773296, 0.66227035, 0.86985725, 0.82311261,
       0.22708294, 0.65643895, 0.39608788, 0.40743591, 0.53118665])

In [15]:
h82 = pd.read_csv("homework_8.2.csv").drop("Unnamed: 0", axis=1)
h82

,X,Y,Z1,Z2
0,1,4.652085,1.764052,2.320015
1,1,2.590221,0.400157,1.292631
2,1,3.898981,0.978738,0.556423
3,1,5.857179,2.240893,2.345607
4,1,3.647489,1.867558,2.095611
...,...,...,...,...
995,1,5.336848,0.412871,0.510622
996,1,0.936869,-0.198399,1.203125
997,0,-1.346745,0.094192,0.252626
998,0,0.414432,-1.147611,-2.289512


In [17]:
import pandas as pd
import numpy as np

# load data
data = pd.read_csv("homework_8.2.csv")

# columns
treatment_col = "X"
outcome_col = "Y"
z_cols = ["Z1", "Z2"]

# split into treated and control
treated = data[data[treatment_col] == 1].reset_index(drop=True)
control = data[data[treatment_col] == 0].reset_index(drop=True)

# 1. build covariance of Z1, Z2 using everyone
z_all = data[z_cols].to_numpy().T   # shape (2, N)
cov_matrix = np.cov(z_all)          # 2x2
inv_cov = np.linalg.inv(cov_matrix) # 2x2 inverse

# 2. pull out arrays
treated_z = treated[z_cols].to_numpy()   # (T, 2)
control_z = control[z_cols].to_numpy()   # (C, 2)
treated_y = treated[outcome_col].to_numpy()
control_y = control[outcome_col].to_numpy()

# 3. match each treated unit to nearest control (Mahalanobis), with replacement
diff_list = []

for i in range(treated_z.shape[0]):
    treated_point = treated_z[i]                  # shape (2,)
    diff_matrix = control_z - treated_point       # shape (C, 2)
    
    # Mahalanobis distance squared for all controls at once
    left = diff_matrix @ inv_cov                  # (C, 2)
    dist_sq = np.sum(left * diff_matrix, axis=1)  # (C,)
    
    best_index = np.argmin(dist_sq)
    matched_control_y = control_y[best_index]
    
    diff_list.append(treated_y[i] - matched_control_y)

# 4. ATE is mean of treated - matched_control
ate = float(np.mean(diff_list))
print("ATE (Mahalanobis matching):", ate)


ATE (Mahalanobis matching): 3.437678997912609


In [19]:
import pandas as pd
import numpy as np

# load data
data = pd.read_csv("homework_8.2.csv")

# columns
treatment_col = "X"
outcome_col = "Y"
z_cols = ["Z1", "Z2"]

# split into treated and control
treated = data[data[treatment_col] == 1].reset_index(drop=True)
control = data[data[treatment_col] == 0].reset_index(drop=True)

# 1. build covariance of Z1, Z2 using everyone
z_all = data[z_cols].to_numpy().T   # shape (2, N)
cov_matrix = np.cov(z_all)          # 2x2
inv_cov = np.linalg.inv(cov_matrix) # 2x2 inverse

# 2. pull out arrays
treated_z = treated[z_cols].to_numpy()   # (T, 2)
control_z = control[z_cols].to_numpy()   # (C, 2)
treated_y = treated[outcome_col].to_numpy()
control_y = control[outcome_col].to_numpy()

# 3. match each treated unit to nearest control (Mahalanobis), with replacement
diff_list = []

for i in range(treated_z.shape[0]):
    treated_point = treated_z[i]                  # shape (2,)
    diff_matrix = control_z - treated_point       # shape (C, 2)
    
    # Mahalanobis distance squared for all controls at once
    left = diff_matrix @ inv_cov                  # (C, 2)
    dist_sq = np.sum(left * diff_matrix, axis=1)  # (C,)
    
    best_index = np.argmin(dist_sq)
    matched_control_y = control_y[best_index]
    
    diff_list.append(treated_y[i] - matched_control_y)

diff_matrix


array([[ 0.04704169, -3.17255936],
       [ 0.09518005, -1.35566744],
       [ 0.64226213, -1.54978431],
       ...,
       [ 0.2925912 , -0.95049837],
       [-0.94921205, -3.49263688],
       [-0.15971518, -2.87220897]], shape=(517, 2))